# Quickstart

In this notebook you will learn the basics of using `p-winds` to model the upper atmosphere (up to many planetary radii) of a H/He-dominated planet, and to compute the steady-state ionization balance of H.

`p-winds` is largely based on the theoretical framework of [Oklopčić & Hirata (2018)](https://ui.adsabs.harvard.edu/abs/2018ApJ...855L..11O/abstract) and [Lampón et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020A%26A...636A..13L/abstract), which themselves based their work on the stellar wind model of [Parker (1958)](https://ui.adsabs.harvard.edu/abs/1958ApJ...128..664P/abstract).

**Notice**: `p-winds` is not suitable for simulations of the lower atmosphere (above $\sim 10^{-7}$ bar)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import astropy.units as u
import astropy.constants as c
from p_winds import tools, parker, hydrogen

# Uncomment the next line if you have a MacBook with retina screen
# %config InlineBackend.figure_format = 'retina'
pylab.rcParams['figure.figsize'] = 9.0,6.5
pylab.rcParams['font.size'] = 18

Let's start by retrieving the planetary and stellar parameters of HD 209458 b. Depending on your versions of `astropy` and `astroquery`, you may get some warnings from those packages. You may also get warnings from `p-winds` asking you to give some attention to a "bad" parameter.

In [ ]:
hd209b_params, hd209_params = tools.fetch_planet_system('HD 209458 b')

In my case I got a warning saying that the planetary parameter `'semi_major_axis'` of HD 209458 b requires my attention. So let's set its value manually.

We will assume that our planet has an isothermal upper atmosphere with temperature of $9 \times 10^3$ K and a total mass loss rate of $8 \times 10^{10}$ g s$^{-1}$ based on the results from [Salz et al. 2016](https://ui.adsabs.harvard.edu/abs/2016A%26A...586A..75S/abstract). We will also assume a H vs. He fraction (in number of atoms) of $0.9$.

In [ ]:
# HD 209458 b
R_pl = hd209b_params['radius'][0]  # Planetary radius
M_pl = hd209b_params['mass'][0]  # Planetary mass
a_pl = 0.04707 * u.au  # Semi-major axis
m_dot = 8E10 * u.g / u.s  # Planetary mass loss rate (assumed)
T_0 = 9E3 * u.K  # Atmospheric temperature (assumed)
d_st = hd209_params['distance'][0]  # Stellar distance
h_he = 0.9  # H/He fraction (assumed)

The next step is to calculate the structure of the planetary atmosphere in terms of densities and velocities.

**Note**: `p-winds` requires the use of `astropy.Quantity` for some of its inputs to avoid errors due to unit conversion. Furthermore, some of the quantities that the code calculates are in "convenience units" to avoid numerical overflows (e.g., velocities and densities calculated by `parker.structure()` are measured in units of sound speed and density at the sonic point, respectively).

In [ ]:
vs = parker.sound_speed(T_0, h_he).to(u.km / u.s)  # Speed of sound (assumed to be constant)
rs = parker.radius_sonic_point(M_pl, vs).to(u.jupiterRad)  # Radius at the sonic point
rhos = parker.density_sonic_point(m_dot, rs, vs).to(u.g / u.cm ** 3)  # Density at the sonic point

# The `parker.structure()` function requires us to pass values of radius in units of 
# radius at the sonic point (`rs`). So, first we build an `r_array` from 1 to 15 
# planetary radii, than change its unit to `rs`
r_array = (np.linspace(1, 15, 500) * R_pl / rs).decompose().value
v_array, rho_array = parker.structure(r_array)

# Convenience arrays for the plots
r_plot = r_array * rs / R_pl
v_plot = v_array * vs
rho_plot = rho_array * rhos

# Finally plot the structure of the upper atmosphere
# The circles mark the velocity and density at the sonic point
ax1 = plt.subplot()
ax2 = ax1.twinx()
ax1.semilogy(r_plot.value, v_plot.value, color='C0')
ax1.plot((rs / R_pl).value, vs.value, marker='o', markeredgecolor='w', color='C0')
ax2.semilogy(r_plot.value, rho_plot.value, color='C1')
ax2.plot((rs / R_pl).value, rhos.value, marker='o', markeredgecolor='w', color='C1')
ax1.set_xlabel(r'Radius (R$_{\rm pl}$)')
ax1.set_ylabel(r'Velocity (km s$^{-1}$)', color='C0')
ax2.set_ylabel(r'Density (g cm$^{-3}$)', color='C1')
ax1.set_xlim(1, 10)
plt.show()

The next step is to calculate the steady-state distribution of H (neutral or ionized) in the atmosphere. To do that, first we need to retrieve the high-energy spectrum of the host star with fluxes at the planet. For convenience, there is a text file in the `data` folder of the `p-winds` package containing the spectrum arriving at HD 209458 b (`HD209458b_spectrum_lambda.dat`). This spectrum was retrieved from the [X-exoplanets](http://sdc.cab.inta-csic.es/xexoplanets/jsp/homepage.jsp) database, the unit of energy was changed from photons to erg, and the flux scaled to the semi-major axis of HD 209458 b.

There is a convenience method in `tools.make_spectrum_dict()` that takes text files as input for the spectrum and transforms it into a `dict` that can be used as input for our calculations.

In [ ]:
units = {'wavelength': u.angstrom, 'flux': u.erg / u.s / u.cm ** 2 / u.angstrom}
spectrum = tools.make_spectrum_dict('../../data/HD209458b_spectrum_lambda.dat',
                                    units)
plt.loglog(spectrum['wavelength'], spectrum['flux_lambda'])
plt.ylim(1E-5, 1E4)
plt.xlabel(r'Wavelength (${\rm \AA}$)')
plt.ylabel(r'Flux density (erg s$^{-1}$ cm$^{-2}$ ${\rm \AA}^{-1}$)')
plt.show()

Finally, we can calculate the distribution of ionized/neutral hydrogen. This involves calculating the differential equation 13 in [Oklopčić & Hirata (2018)](https://ui.adsabs.harvard.edu/abs/2018ApJ...855L..11O/abstract). To achieve this, we start from an initial state at the outermost layer of the atmosphere, and integrate inwards. We utilize the `hydrogen.ion_fraction()` function. It takes as input many of the parameters we already set above. 

One thing that you may want to change is the `initial_state` of the integration, which is an optional parameter. The initial state is the `y0` of the differential equation to be solved. This array has two items: the initial value of `f_ion` (ionization fraction) and `tau` (optical depth) at the outer layer of the atmosphere. The standard value for this parameter is `numpy.array([1.0, 0.0])`, i.e., completely ionized at the outer layer and with null optical depth.

**Note**: This integration may take some time to compute depending on the system you are modeling and the number of points you are sampling the radius, and of course depending on your computer. 

**Note 2**: You may run into numerical overflows here because the optical depth can increase by several orders of magnitude near the surface of the planet. If this becomes an issue, you can make calculations starting a bit above the planetary surface (e.g., $1.5$ instead of $1.0$ R$_\mathrm{pl}$).

In [ ]:
# We compute `f_ion` from 1 to 15 planetary radii
r = np.linspace(1.0, 15, 500)
initial_state = np.array([1.0, 0.0])

f_r, tau_r = hydrogen.ion_fraction(r, R_pl, T_0, h_he, 
                                         m_dot, M_pl, spectrum,
                                         initial_state)

And now we plot the result.

In [ ]:
f_ion = f_r
f_neutral = 1 - f_r

plt.plot(r, f_neutral, color='C0', label='f$_\mathrm{neutral}$')
plt.plot(r, f_ion, color='C1', label='f$_\mathrm{ion}$')
plt.xlabel(r'Radius (R$_\mathrm{pl}$)')
plt.ylabel('Number fraction')
plt.xlim(1, 10)
plt.ylim(0, 1)
plt.legend()
plt.show()